 ---
# **Title**:TRABAJO AP. DDPG TD(3) (Gradiente de política determinista profunda (TD3) de doble retardo. Entornos de estados continuos: Humanoide). + Conexión Chatgpt (o similar)  + modelo dalle (similr)
# **Author**: José Javier Gutiérrez Gil
# **Date**: 2024-02-18
# ***Univeridad de Valencia. Grado de Ciencia de Datos***
---

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/td3')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/td3')

# Pasos del código:

1.  Construir la **memoria de la repetición de experiencias**

2.  Construir una red neuronal para el **actor del modelo** y una red neuronal para el **actor del objetivo (Target)**

3. Construir dos redes neuronales para los dos **críticos del modelo** y dos redes neuronales para los dos ***críticos del objetivo (Target)**

4. Construir **clase TD3** que contendra el **entrenamiento del modelo**

5. **Inicializamos** los **hiperparámetros**

6. **Creamos las carpetas y nombres de fichero** donde se almacenarán los modelos y su entrenamiento.

7. **Cargamos el entorno deseado de gym** y obtenemos información de sus estados y acciones

8. **Entrenamos el entorno** Humanoide  con el DDPG TD3

9. **Leemos los ficheros almacenados en disco** de los modelos y el entrenamient, y **creamos los videos del entrenamiento**



## **Item (-1)**: Instalación de los paquetes necesarios par ala ejecición del *entorno*


In [ ]:
!pip install pybullet
!pip install gym==0.22.0 # Versión más actual que contiene la calse Monitor y así poder crear los videos del entrenamiento. Al monos en la 0.23 me da error

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.22.0-py3-none-any.whl size=708365 sha256=d3f471d004b4857f4d98b9e3943e22a1406707c51cf6b2e76b75b1df99b089c1
  Stored in directory: /root/.cache/pip/wheels/42/e8/e8/6dfbc92a1dcd76c1a5e2bb982750fd6b7e792239f46039e6b1
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


In [ ]:
!pip install -U 'mujoco-py<2.2,>=2.1




/bin/bash: -c: line 1: unexpected EOF while looking for matching `''
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [ ]:
# !pip install mujoco-py #para poder crear otros entornos con simulaciones físicas pero me da error en google colab luego el hacer import


##  **Step 0:** Cargamos las librerías utilizadas en todo el código


In [ ]:
import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#####
import gym
import pybullet_envs
from gym import wrappers
##
import torch
import torch.nn as nn
#####
from TD3 import TD3, ReplayBuffer
from TD3 import created_models_directory, mkdir
from TD3 import evaluate_train_policy, evaluate_policy
from TD3 import noisy_action_wrapper, save_env, load_env, create__metrics_imagen
from TD3 import serialize_object, lists_to_serializable_object, serialize_training

# **Step 1:** Inicializamos los hiperparámetros del modelo e implementación

In [ ]:

env_name            =  "HumanoidBulletEnv-v0"  # Nombre del entorno que vamos a entrena. Con forme se ha implementado el modelo TD3 poría ser cuaalquier entorno (preferiblemente de estados de acciones continuos)
seed                = 0        # Semilla utilizada para garantizar la reproducibilidad de los resultados
initial_memory_prob = 0.2      # Define la probabilidad inicial de tomar una acción de la memoria de repetición
max_start_steps     = 1e4      # Define el número máximo de pasos iniciales
#start_steps         = 1e4#1e4  --123-- lo modificamos por un decaimiento según los pasos de entrenamiento # Número de iteraciones/timesteps antes de que el modelo comience a utilizar la red de políticas en lugar de elegir acciones al azar
eval_frequency      = 5e3      # Frecuencia de evaluación, es decir, cada cuántos pasos/timesteps se evalúa el desempeño del modelo
max_timesteps       = 1e6      # Número máximo de iteraciones/timesteps permitidos
save_models         = True     # Booleano que indica si se deben guardar los modelos pre-entrenados o no
max_explore_noise   = 0.08      # Desviación estándar del ruido gaussiano utilizado para la exploración...--123-- CaMBIO  0.01 por 0.2 y creo un current_noise_explore dependiendo del steo en el que estamos
batch_size          = 100      # Tamaño del lote de muestras utilizadas en cada iteración de entrenamiento
gamma               = 0.99     # Factor de descuento gamma que afecta la importancia de las recompensas futuras en la función de pérdida
target_update_freq  = 0.005    # Tasa de actualización para suavizar los parámetros de la red objetivo
policy_noise        = 0.2      # Desviación estándar del ruido gaussiano agregado a las acciones para promover la exploración
noise_clip          = 0.5      # Valor máximo permitido para el ruido gaussiano agregado a las acciones (política)
policy_freq         = 2        # Número de iteraciones entre actualizaciones de la red de políticas (modelo actor)

# **Step 2:** Creamos los directorios y cargamos el entorno de trabajo

In [ ]:
#Creamos los directorios de almacenamento de resultados y métricas
file_model_name = created_models_directory (env_name, seed, save_models, "v0_1")

# Cargamos el entorno sobre el cual ejecutaremso el modelo DDPG TD3
###############################################################################
##  CARGAMOS EL ENTORNO HUMANOIDE V0 Y LE INTRODUCIMOS EL WRAPPER PARA AÑADIR RUIDO A
##  LAS ACCIONES OBTENIDAS POR EXPLORACIÓN.
################################################################################
env = gym.make (env_name)
max_episode_steps = env._max_episode_steps
env = noisy_action_wrapper (env, noise_level = policy_noise) #le agregamos ruido a la accion obtenida del entorno para darle ms estabilidad al entrenamiento
# Fijamos la semilla y obtenemos información del entorno (Estados, acciones)
env.seed (seed)
torch.manual_seed (seed)
np.random.seed (seed)

# Limitación de las acciones
state_dim  = env.observation_space.shape [0]
action_dim = env.action_space.shape [0]
max_action = float(env.action_space.high [0])

#Creamos los directorios de trabajo donde guardará los videos del entrenamiento
work_dir          = mkdir ('exp', 'brs')
monitor_dir       = mkdir (work_dir, 'monitor')

save_env_vid      = False
if save_env_vid:
  env = wrappers.Monitor (env, monitor_dir, force = True)
  env.reset()

---------------------------------------
Fichero de los modelos entrenados: TD3_HumanoidBulletEnv-v0_0_v0_1
---------------------------------------


# **Step 3:** Proceso de entrenamiento


**Entrenamiento con selección de acción adaptativa:**

Durante el entrenamiento, se utiliza un enfoque de selección de acción adaptativa que combina la exploración y la explotación para mejorar la capacidad del agente para aprender y mejorar su política a lo largo del tiempo.

La función select_action_ se encarga de seleccionar la acción para el agente, ya sea del entorno real o de la experiencia del agente, dependiendo de varios factores, como el número total de pasos, los pasos iniciales y la probabilidad de tomar una acción de la experiencia.

- Exploración:
  - Durante los primeros pasos (total_steps < start_steps), el agente explora el entorno tomando acciones al azar.
  - Luego de estos pasos iniciales, se reduce gradualmente la probabilidad de exploración para comenzar a confiar más en el conocimiento adquirido hasta el momento.

- Explotación:
   - Después de superar los primeros pasos, el agente comienza a utilizar su política aprendida con una probabilidad. Sin embargo, aún se mantiene una pequeña probabilidad de exploración para evitar el estancamiento y permitir una mejora continua de la política. Durante el entrenamiento, se registran las recompensas, las pérdidas y el factor de exploración promedio para cada episodio. Estos datos se utilizan para analizar y visualizar el rendimiento del agente a lo largo del tiempo.
   
   Al final del entrenamiento, se guarda el modelo entrenado y se generan gráficos para mostrar las métricas de entrenamiento, como la recompensa promedio, la pérdida promedio y el factor de exploración promedio.

In [ ]:
def calc_memory_prob (total_steps, start_steps, initial_memory_prob):
    """
    Define una función para calcular la probabilidad de tomar una acción de la memoria de repetición en función
    del número total de pasos.

    Args:
        total_steps (int): El número total de pasos realizados en el entrenamiento.
        start_steps (int): El número de pasos antes de comenzar a utilizar la red de políticas en lugar de la
                           memoria de repetición.

    Returns:
        float: La probabilidad de tomar una acción de la memoria de repetición en el rango [0, 1].
    """
    if total_steps < start_steps:
        # Probabilidad decreciente de tomar una acción de la memoria de repetición a medida que total_steps aumenta
        return initial_memory_prob * (1 - total_steps / start_steps)
    else:
        # Probabilidad creciente de tomar una acción de la memoria de repetición a medida que total_steps supera start_steps
        return initial_memory_prob + (1 - initial_memory_prob) * (total_steps - start_steps) / (max_timesteps - start_steps)

def select_action (policy, env, total_steps, start_steps, memory_prob, obs):
    """
    Selecciona una acción para el agente, ya sea del entorno real o de la experiencia del agente,
    en función del número total de pasos, los pasos iniciales y la probabilidad de tomar una acción
    de la experiencia. Hay que tener en cuetna la difrencia entre exploracion Vs explotación:

    * Exploración: Se refiere a probar acciones desconocidas para descubrir información sobre
      el entorno y mejorar la política del agente.

    * Explotación: Se refiere a aprovechar al máximo el conocimiento adquirido hasta el momento,
      es decir, elegir las acciones que el agente considera óptimas según su conocimiento actual.

      Antes de los primeros pasos (total_steps < start_steps):
        El agente selecciona acciones aleatorias del entorno real para explorar el espacio de acciones.
      Después de los primeros pasos (total_steps >= start_steps):
        El agente elige aleatoriamente entre la memoria de repetición (explotación) y el entorno real (exploración) en función de la probabilidad memory_prob.
        Si se elige la memoria de repetición, se selecciona una acción del modelo entrenado y se aplica ruido a la acción para fomentar una exploración más refinada.

    Args:
        policy      (TD3): plotica a entrenar
        env         (gym): entorno sobre el cual se aplica el modelo
        total_steps (int): El número total de pasos que ha realizado el agente.
        start_steps (int): El número de pasos iniciales antes de que el agente comience a utilizar su política.
        memory_prob (float): La probabilidad de tomar una acción de la experiencia del agente.
        obs (array): La observación actual del entorno.

    Returns:
        array: La acción seleccionada para ser ejecutada en el entorno.
    """
   #print (f'total_steps:{total_steps} - tart_steps:{start_steps}  ')

    # Tomar acciones aleatorias antes de alcanzar el número de pasos iniciales
    if total_steps < start_steps:
        # Si estamos en los primeros pasos, acceder al entorno real (exploración)
        action = env.action_space.sample()
    # Después de superar los primeros pasos, usar el modelo entrenado con una probabilidad inversamente proporcional
    # a la probabilidad de tomar una acción de la experiencia
    else:
        # Generar un número aleatorio para determinar si tomar una acción de la experiencia del agente
        if np.random.rand() < memory_prob:
            # Tomar una acción de la experiencia del agente (exploración)
            action = policy.select_action(np.array(obs))
            # Aplicar ruido a la acción para fomentar la exploración de forma más refinada
            current_explore_noise = max_explore_noise * max(1 - total_steps / max_timesteps, 0)
            if current_explore_noise != 0:
                action = (action + np.random.normal(0, max_explore_noise, size=action_dim)).clip(env.action_space.low, env.action_space.high)
        else:
            # Tomar una acción aleatoria del entorno real (exploración)
            action = env.action_space.sample()
    return action

# Creamos el objeto DDPG TD3 con los hiperparámetros definidos anteriormente
policy = TD3 (state_dim, action_dim, max_action, max_timesteps, initial_lr =  1e-4) # Notar que realmente el objeto entrenado definira la politica a seguir por el agente

# Nos creamos la memoroa de repetición de experiencias
replay_buff = ReplayBuffer ( max_capacity = 2e4) # Notar que la máxima  capacidad de la memoria por defecto es: max_capacity = 1e4
# replay_buff = SimplePrioritizedReplayBuffer ()
# replay_buff = PrioritizedReplayBuffer ( max_capacity = int(1e10))
# Lista donde se guardarán las evaluaciones de la política durante el entrenamiento
evaluations = [evaluate_train_policy (policy, env)]

# Inicialización de variables utilizadas en el entrenamiento
explore_prob_steps  = 0
total_steps         = 0     # Número total de pasos de entrenamiento
episode_steps       = 0     # Número de pasos realizados (número de episodios)
steps_since_eval    = 0     # Número de pasos desde la última evaluación de la política
episode_num         = 0     # Número de episodios completados durante el entrenamiento
done                = True  # Indica si el episodio actual ha finalizado

t0 = time.time()  # Tiempo inicial de referencia para medir el tiempo de entrenamiento
n_steps_epochs = []
all_rewards    = []
avg_losses_c   = []
avg_losses_a   = []
all_losses_c   = []
all_losses_a   = []
target_qs_c1   = []
target_qs_c2   = []
target_qs      = []
all_exploration_factor =  []

############################################################################
# step 0: Bucle principal del entrenameinto de nuestro modelo.
##  REalizamos tantos pasos como indica el hiperparámetro "max_timesteps"
## A no ser que se estanque en un maximo local (mínimo) el humanoide aprenderá
## mejor cuantos más pasos realiza. Esta implementacición V1 utiliza una estrategia
## balanceada de exploracion y explotación pero con decaimiento en la capacida de
## explorar una vez pasamos a explotar.
#############################################################################

while total_steps < max_timesteps: #son pasos de tiempo
    # step 1: Se comprueba si el episodio ha concluido. Si lo ha hehco, realiamso el entrenamiento del modelo
    # con las acciones almacenadas en ReplayBuffer
    if done:
        # Comienza el entrenamiento del modelo si no es la primera iteración
        if total_steps != 0:

            # , expl_fact
            print (f'Número de pasos del episodeo {episode_num} son episode_steps:{episode_steps}')
            n_steps_epochs.append (episode_steps)
            rewards, losses_c,losses_a, target_qs_critic1, target_qs_critic2, target_qs = policy.train (replay_buff, episode_steps, batch_size, gamma,
                                                                                                          target_update_freq, policy_noise, noise_clip, policy_freq)
            #############
            # Nos guardamos las métricas del entrenamiento a estudio
            #############
            avg_reward    = np.mean (rewards)
            avg_loss_c    = np.mean (losses_c)
            avg_loss_a    = np.mean (losses_a)
            #avg_expl_fact = np.mean (expl_fact)

            all_rewards.append (avg_reward)
            avg_losses_c.append (avg_loss_c)
            avg_losses_a.append (avg_loss_a)
            all_losses_c.append (losses_c.copy())
            all_losses_a.append (losses_a.copy())

            target_qs_c1.append (target_qs_critic1.copy())
            target_qs_c2.append (target_qs_critic2.copy())
            target_qs.append (target_qs.copy())

            #all_exploration_factor.append (avg_expl_fact)

            print ("Total Steps: {} Episode Num: {} Reward: {} avg_loss_c: {} avg_loss_a: {}".format(total_steps, episode_num, episode_reward, avg_loss_c, avg_loss_a))

        # step 2: se evalúa el rendimiento del episodio actual y se guarda la política
        # si se cumplen ciertas condiciones o criterios predefinidos.
        if steps_since_eval >= eval_frequency:
            steps_since_eval %= eval_frequency
            evaluations.append (evaluate_train_policy (policy, env))
            policy.save (file_model_name, directory="./pytorch_models")
            np.save("./results/%s" % (file_model_name), evaluations)
            save_env (file_model_name, directory = "./results", env = env) # Guardamos el entorno en el estado actual
            # Serializar el ReplayBufferMemory y guardarlo en un archivo
            serialize_object (replay_buff, './results/replay_buffer_memory_v0_1.pickle')
            # Serializamos las metricas del entrenamiento
            lists_train_metrics = [all_rewards, avg_losses_c,avg_losses_a, all_losses_c, all_losses_a]
            attribute_names     = ['rewards', 'losses']
            lists_to_serializable_object (lists_train_metrics, attribute_names, './results/serialized_list_train_metrics_V0_1.pickle')

        # step 3: Reiniciar el entorno cuando finaliza el episodio de entrenamiento
        # en el primer ciclo es el primer paso que se realiza
        obs = env.reset ()

        # Establecer "done" a Falso para parar el episodeo
        done = False

        # step 4: Restablecer la recompensa del episodio y el contador de pasos del episodio
        episode_reward = 0
        episode_steps  = 0
        episode_num += 1

    # step 4: Seleccionamso la acción bien sea pro exploración o explotación
    start_steps = max_start_steps * max(1 - total_steps / max_timesteps, 0)  # Decaimiento lineal
    memory_prob = calc_memory_prob (total_steps, start_steps, initial_memory_prob)                # Probabilidad de tomar la acción en otro entorno
    action      = select_action (policy, env, total_steps, start_steps, memory_prob, obs)

    # step 5: El agente ejecuta una acción en el entorno, lo que resulta en una transición
    # de estado. Además, el agente recibe una recompensa del entorno como resultado
    # de su acción. Esta acción puede cambiar el estado del entorno y, por lo tanto,
    # influir en las futuras observaciones y recompensas del agente.
    #print (f' * nueva action:{action} -- tamaño:{len (action)}')
    new_obs, reward, done, _ = env.step(action)
    # print (f' * nuevo reward:{reward}')
    # Comprueba si el episodio ha terminado
    done_bool = 0 if episode_steps + 1 == max_episode_steps else float(done)

    # Aumenta la recompensa total del episodio
    episode_reward += reward

    # step 6: Almacenar nueva transición en el búfer de repetición de experiencias
    replay_buff.add ((obs, new_obs, action, reward, done_bool))
    #error_explora = 0.01 # añadimos un error muy bajo para asegurarnos que almenos se utiliza una vez
    #replay_buff.add (error_explora, (obs, new_obs, action, reward, done_bool))

    # Actualizar estado, tiempo de paso del episodio, tiempo total de pasos y pasos desde la última evaluación de la política
    obs = new_obs
    episode_steps    += 1
    total_steps      += 1
    steps_since_eval += 1
    create__metrics_imagen (evaluations, all_rewards,  avg_losses_c,avg_losses_a, all_losses_c,
                                                    all_losses_a, target_qs_c1,
                                                    target_qs_c2, target_qs,
                                                    n_steps_epochs,  "v0_1", episode_num, total_steps  )
tf = time.time()  # Tiempo final de referencia para medir el tiempo de entrenamiento
serialize_training (t0, tf, total_steps,"v0_1")

# Añadimos la última actualización de la política a la lista de evaluaciones previa y guardamos nuestro modelo
evaluations.append (evaluate_train_policy (policy, env))
if save_models:
  policy.save ("%s" % (file_model_name), directory="./pytorch_models")
np.save ("./results/%s" % (file_model_name), evaluations)

Se han truncado las últimas 5000 líneas del flujo de salida.
Número de pasos del episodeo 8660 son episode_steps:19
Total Steps: 210293 Episode Num: 8660 Reward: -42.19167066192163 avg_loss_c: 174961.79975328947 avg_loss_a: -4033.9985094572367
Número de pasos del episodeo 8661 son episode_steps:26
Total Steps: 210319 Episode Num: 8661 Reward: -51.3532869242665 avg_loss_c: 169784.8759014423 avg_loss_a: -4081.791278545673
Número de pasos del episodeo 8662 son episode_steps:25
Total Steps: 210344 Episode Num: 8662 Reward: -53.719907825310386 avg_loss_c: 171626.44375 avg_loss_a: -4139.225166015625
Número de pasos del episodeo 8663 son episode_steps:22
Total Steps: 210366 Episode Num: 8663 Reward: -51.64617052486093 avg_loss_c: 162431.6044034091 avg_loss_a: -4182.972500887784
Número de pasos del episodeo 8664 son episode_steps:43
Total Steps: 210409 Episode Num: 8664 Reward: -51.066421457418656 avg_loss_c: 170899.12590843023 avg_loss_a: -3992.4847099836484
Número de pasos del episodeo 8665 

KeyboardInterrupt: 

# **Step 4:** Evaluación de la política para extraeer videos  de su rendimiento

In [ ]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs
import gym
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque
#import mujoco_py
from TD3 import TD3, evaluate_policy, created_models_directory
import pybullet as p

# Iniciar el servidor de física de PyBullet
p.connect(p.DIRECT)



#######################################
###
######################################
env_name           = "HumanoidBulletEnv-v0"  # Nombre del entorno que vamos a entrena. Con forme se ha implementado el modelo TD3 poría ser cuaalquier entorno (preferiblemente de estados de acciones continuos)
seed               = 0        # Semilla utilizada para garantizar la reproducibilidad de los resultados
start_steps        = 1e4      # Número de iteraciones/timesteps antes de que el modelo comience a utilizar la red de políticas en lugar de elegir acciones al azar
eval_frequency     = 5e3      # Frecuencia de evaluación, es decir, cada cuántos pasos/timesteps se evalúa el desempeño del modelo
max_timesteps      = 5e12     # Número máximo de iteraciones/timesteps permitidos
save_models        = True     # Booleano que indica si se deben guardar los modelos pre-entrenados o no
explore_noise      = 0.1      # Desviación estándar del ruido gaussiano utilizado para la exploración
batch_size         = 100      # Tamaño del lote de muestras utilizadas en cada iteración de entrenamiento
gamma              = 0.99     # Factor de descuento gamma que afecta la importancia de las recompensas futuras en la función de pérdida
target_update_freq = 0.005    # Tasa de actualización para suavizar los parámetros de la red objetivo
policy_noise       = 0.2      # Desviación estándar del ruido gaussiano agregado a las acciones para promover la exploración
noise_clip         = 0.5      # Valor máximo permitido para el ruido gaussiano agregado a las acciones (política)
policy_freq        = 2        # Número de iteraciones entre actualizaciones de la red de políticas (modelo actor)

env_name = "HumanoidBulletEnv-v0"
seed = 0

work_dir = os.path.join('exp', 'brs')
monitor_dir = os.path.join(work_dir, 'monitor')


file_model_name = created_models_directory (env_name, seed, save_models, "v0_1")
print ("---------------------------------------")
print ("Configuración: %s" % (file_model_name))
print ("---------------------------------------")

eval_episodes = 1000000
save_env_vid  = True

env = gym.make (env_name)
max_episode_steps = env._max_episode_steps
if save_env_vid:
  env = wrappers.Monitor (env, monitor_dir, force = True)
  # env = RecordEpisodeStatistics (env)
  env.reset ()
env.seed (seed)

torch.manual_seed (seed)
np.random.seed (seed)

state_dim  = env.observation_space.shape [0]
action_dim = env.action_space.shape [0]
max_action = float (env.action_space.high [0])

policy = TD3 (state_dim, action_dim, max_action,max_timesteps, initial_lr =  1e-4) # Notar que realmente el objeto entrenado definirá la política a seguir por el agente
policy.load (file_model_name, './pytorch_models/')
avg_reward, episode_rewards = evaluate_policy (policy, env, eval_episodes = eval_episodes)

# Graficar las recompensas por episodio
plt.plot(episode_rewards)
plt.xlabel('Episodio')
plt.ylabel('Recompensa')
plt.title('Recompensa por Episodio durante la Evaluación')

# Guardar la gráfica en un archivo de imagen (por ejemplo, en formato PNG)
plt.savefig('./results/recompensas_por_episodio_evaluacion_v0_1.png')

# Mostrar la gráfica en la ventana
plt.show()
